# Lasso and Ridge Regression Implementation
In this Notebook, we implement Lasso (L1 Regularization) and Ridge regression (L2 regularization) from scratch, and compare it with the pre-defined models from the Scikit-learn library.

### Initial functions

In [16]:
# Importing libraries 
import numpy as np
import pandas as pd

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Most formulations are same in both Lasso and Ridge regression. From an implementation perspective, they differ solely in the regularization term. Hence, we implement a common regression class, in which L1 or L2 regularization can be chosen as a parameter.

In [17]:
class Regression:
    
    def __init__(self, regularization, lr, epoch):
        self.m = None #samples
        self.n = None #features
        self.w = None #weight
        self.b = None #bias
        self.regularization = regularization #penalty object
        self.lr = lr #learning rate
        self.epoch = epoch #iteration
        
    def __calculate_cost(self, y, y_pred):
        return (1 / (2*self.m)) * np.sum(np.square(y_pred-y)) + self.regularization(self.w)
    
    def __hypothesis(self, w, X):
        return np.dot(X, w) 
    
    def __initialization(self, X):
        X = np.insert(X, 0, 1, axis=1)
        self.m, self.n = X.shape
        self.w = np.zeros((self.n,1))
        return X
    
    def __update_parameters(self, X, y, y_pred):
        dw = (1/self.m) * np.dot(X.T, (y_pred - y)) + self.regularization.derivation(self.w)
        self.w = self.w - self.lr * dw
        return True
        
    def fit(self, X, y):
        X = self.__initialization(X)
        for e in range(1, self.epoch+1):
            y_pred = self.__hypothesis(self.w, X)
            cost = self.__calculate_cost(y, y_pred)
            self.__update_parameters(X, y, y_pred)
            if e % 100 == 0:
                print(f"The Cost in iteration {e}----->{cost} :)")
        return True

    def predict(self, X_test):
        X_test = np.insert(X_test, 0 , 1, axis= 1)
        y_pred = self.__hypothesis(self.w, X_test)
        return y_pred

Define the regularization terms for Lasso and Ridge respectively:

In [18]:
class LassoPenalty:
    
    def __init__(self, l):
        self.l = l # lambda value
        
    def __call__(self,w):
        return self.l * np.sum(np.abs(w))
        
    def derivation(self, w):
        return self.l * np.sign(w)
    
class RidgePenalty:
    
    def __init__(self, l):
        self.l = l
        
    def __call__(self, w):
        return self.l * np.sum(np.square(w))
        
    def derivation(self, w):
        return self.l * 2 * w

In [19]:
class Lasso(Regression):
    
    def __init__(self, l, lr, epoch):
        self.regularization = LassoPenalty(l)
        super().__init__(self.regularization, lr, epoch )
        
class Ridge(Regression):
    
    def __init__(self, l, lr, epoch):
        self.regularization = RidgePenalty(l)
        super().__init__(self.regularization, lr, epoch )

### Dataset Operations

In [20]:
# Importing dataset     
df = pd.read_csv( "../ML-Project-CS361/cleaned_shifted_data.csv" ) 
drop_cols = [0,1,2,12,14,16]
drop_cols = df.columns[drop_cols]
drop_cols # Dropping unecessary columns

Index(['Timestamp', 'Unnamed: 0', 'Station', 'Checks', 'AQI_bucket_calculated',
       'AQI_bucket_calculated_shifted'],
      dtype='object')

In [21]:
# Drop the columns and make the changes in-place
df.drop(columns=drop_cols, inplace=True)

In [22]:
print(df.shape)  # The dataset has a size of 1,74,762 records, 10 features, 1 target variable

(174762, 11)


In [23]:
df.head() 

,PM2.5 (µg/m³),PM10 (µg/m³),NO (µg/m³),NO2 (µg/m³),NOx (ppb),NH3 (µg/m³),SO2 (µg/m³),CO (mg/m³),Ozone (µg/m³),AQI_calculated,AQI_calculated_shifted
0,46.00,80.00,1.29,9.16,12.02,27.19,13.56,0.40,15.80,67.0,296.0
1,46.00,80.00,1.74,8.93,12.48,30.29,13.71,0.41,15.52,68.0,297.0
2,45.62,79.92,1.87,8.56,12.17,28.20,13.88,0.41,15.33,68.0,298.0
3,41.00,72.92,1.83,8.72,12.37,26.69,13.77,0.40,15.30,68.0,298.0
4,41.00,79.00,1.69,7.91,11.30,26.83,13.87,0.41,15.49,68.0,299.0


Since the original dataset is too large to fit into a numpy array, take a random subset of this data. 

In [24]:
# Randomly sample 32000 rows
df_subset = df.sample(n=32000, random_state=42)
print(df_subset.shape)

(32000, 11)


In [25]:
df_subset.describe()

,PM2.5 (µg/m³),PM10 (µg/m³),NO (µg/m³),NO2 (µg/m³),NOx (ppb),NH3 (µg/m³),SO2 (µg/m³),CO (mg/m³),Ozone (µg/m³),AQI_calculated,AQI_calculated_shifted
count,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000
mean,60.007376,111.450673,9.785336,8.365729,17.911769,9.936954,18.552968,0.689827,24.399094,140.854719,138.908719
std,60.077518,111.412286,21.020019,10.607242,34.041483,7.555035,13.777714,0.623949,22.927234,104.175769,105.049152
min,0.100000,1.700000,0.010000,0.020000,0.230000,0.010000,0.100000,0.000000,0.020000,14.000000,11.000000
25%,20.000000,37.540000,1.210000,2.520000,4.610000,4.000000,10.120000,0.310000,12.750000,56.000000,55.000000
50%,39.500000,73.105000,3.360000,4.170000,5.470000,7.140000,14.510000,0.490000,18.100000,101.000000,100.000000
75%,81.000000,149.000000,5.610000,10.210000,13.400000,15.310000,22.880000,0.840000,26.760000,216.000000,204.000000
max,916.670000,1000.000000,299.640000,120.200000,330.290000,76.790000,194.130000,6.220000,193.570000,1109.000000,1105.000000


In [26]:
# Separating the features and labels/target variables
X = df_subset.drop('AQI_calculated_shifted',axis = 1)  # feature set
Y = df_subset['AQI_calculated_shifted'] # target variable
print(X.shape)
print(Y.shape)

(32000, 10)
(32000,)


In [27]:
X # it is a pandas dataframe

,PM2.5 (µg/m³),PM10 (µg/m³),NO (µg/m³),NO2 (µg/m³),NOx (ppb),NH3 (µg/m³),SO2 (µg/m³),CO (mg/m³),Ozone (µg/m³),AQI_calculated
56935,5.00,11.00,3.83,4.06,4.83,5.30,27.65,0.35,39.81,47.0
88621,37.71,67.00,5.59,2.55,5.29,6.74,7.27,0.55,28.90,188.0
101450,48.00,129.00,5.45,2.87,5.46,4.05,12.50,0.45,0.37,86.0
30673,31.00,65.00,0.60,4.83,6.13,14.83,12.58,0.20,21.08,219.0
61777,34.22,65.00,4.07,3.90,4.94,4.21,29.36,0.55,21.62,107.0
...,...,...,...,...,...,...,...,...,...,...
15902,18.00,30.46,2.23,6.60,10.70,4.96,12.12,0.33,14.56,29.0
172402,75.00,276.00,0.70,18.30,10.30,7.20,7.10,0.62,49.50,152.0
5603,156.00,299.00,25.50,52.73,98.01,12.33,16.70,2.67,1.37,223.0
22494,88.00,145.00,63.27,11.09,115.64,13.09,11.11,0.94,17.74,135.0


In [28]:
Y # it is a pandas series

56935      35.0
88621     271.0
101450     81.0
30673     214.0
61777     124.0
          ...  
15902      69.0
172402    154.0
5603       95.0
22494     204.0
56153      36.0
Name: AQI_calculated_shifted, Length: 32000, dtype: float64

In [29]:
Y = Y.values.reshape(-1, 1) # convert to a numpy array
Y 

array([[ 35.],
       [271.],
       [ 81.],
       ...,
       [ 95.],
       [204.],
       [ 36.]])

In [30]:
# Splitting dataset into train and test set 
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size = 1 / 3, random_state = 0 )
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(21333, 10)
(10667, 10)
(21333, 1)
(10667, 1)


In [31]:
Y_train

array([[114.],
       [ 64.],
       [323.],
       ...,
       [315.],
       [197.],
       [ 95.]])

In [32]:
# Standardize the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[-0.7578099 , -0.84412827, -0.39188338, ..., -0.41370701,
         0.09591132, -0.70718809],
       [ 1.77848328,  2.02302695,  3.85916975, ...,  4.80450109,
        -1.03646199,  1.08175379],
       [-0.31818575, -0.40355122, -0.20417454, ..., -0.78299558,
         0.85038999,  1.72271158],
       ...,
       [ 1.82920914,  1.19688825, -0.27666568, ...,  1.17583946,
        -0.36280472,  2.02884067],
       [ 0.32434186,  0.28904352, -0.19649336, ...,  0.21247797,
         0.18590704,  1.06262073],
       [-0.96071335, -0.88815873, -0.45765348, ..., -0.76693955,
         0.00809997, -1.02288371]])

### L1 Regularization (Lasso)

Here we can observe much lower values of the cost function, since the regularization term depends only on the absolute values of weights.

In [33]:
parameters = {
    "l" : 0.5,
    "lr" : 0.1,
    "epoch" : 2000
}
model = Lasso(**parameters)
model.fit(X_train, Y_train) 

Y_pred = model.predict(X_test)
score = r2_score(Y_test, Y_pred)  # calculate the R-squared error
print("r2 score", score)  
mse = mean_squared_error(Y_test, Y_pred)  # calculate the mean-squared error
print("MSE", mse)

The Cost in iteration 100----->3386.1369516237537 :)
The Cost in iteration 200----->3383.7526008183654 :)
The Cost in iteration 300----->3383.397821264799 :)
The Cost in iteration 400----->3383.313162705939 :)
The Cost in iteration 500----->3383.2918077991285 :)
The Cost in iteration 600----->3383.286372038173 :)
The Cost in iteration 700----->3383.284986307505 :)
The Cost in iteration 800----->3383.2846329564727 :)
The Cost in iteration 900----->3383.284542850824 :)


The Cost in iteration 1000----->3383.284519873434 :)
The Cost in iteration 1100----->3383.284514014078 :)
The Cost in iteration 1200----->3383.284512519912 :)
The Cost in iteration 1300----->3383.284512138891 :)
The Cost in iteration 1400----->3383.2845120417287 :)
The Cost in iteration 1500----->3383.284512016952 :)
The Cost in iteration 1600----->3383.2845120106335 :)
The Cost in iteration 1700----->3383.2845120090224 :)
The Cost in iteration 1800----->3383.2845120086117 :)
The Cost in iteration 1900----->3383.2845120085067 :)
The Cost in iteration 2000----->3383.28451200848 :)
r2 score 0.4121772777353133
MSE 6603.5127358723275


### L2 Regularization (Ridge)

Here, we can observe much higher values of the cost function, as the regularization term depends on squares of weights.

In [34]:
parameters = {
    "l" : 0.001,
    "lr" : 0.1,
    "epoch" : 2000
}
model = Ridge(**parameters)
model.fit(X_train, Y_train)
 
Y_pred = model.predict(X_test)
score = r2_score(Y_test, Y_pred)
print("r2 score", score)
mse = mean_squared_error(Y_test, Y_pred)
print("MSE", mse)

The Cost in iteration 100----->3276.5546610540678 :)
The Cost in iteration 200----->3271.3805804512185 :)
The Cost in iteration 300----->3270.477918998831 :)
The Cost in iteration 400----->3270.2657072906422 :)
The Cost in iteration 500----->3270.2140746882187 :)
The Cost in iteration 600----->3270.20144192703 :)
The Cost in iteration 700----->3270.1983482514293 :)
The Cost in iteration 800----->3270.197590515133 :)
The Cost in iteration 900----->3270.197404917466 :)
The Cost in iteration 1000----->3270.1973594575347 :)
The Cost in iteration 1100----->3270.1973483226598 :)
The Cost in iteration 1200----->3270.197345595303 :)
The Cost in iteration 1300----->3270.1973449272687 :)
The Cost in iteration 1400----->3270.197344763642 :)
The Cost in iteration 1500----->3270.197344723563 :)
The Cost in iteration 1600----->3270.1973447137466 :)
The Cost in iteration 1700----->3270.1973447113423 :)
The Cost in iteration 1800----->3270.197344710753 :)
The Cost in iteration 1900----->3270.197344710

### Library Implementation

Using Sklearn library implementation of Lasso:

In [35]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
parameters = {'alpha': 0.5,
              'tol': 0.1
              }
model = Lasso(**parameters)

model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
score = r2_score(Y_test, Y_pred)
print("R2 {}".format(score))
mse = mean_squared_error(Y_test, Y_pred)
print("MSE", mse)

R2 0.4118637472264115
MSE 6607.034890818365


Using Sklearn library implementation of Ridge:

In [36]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
parameters = {'alpha': 0.5
              }
model = Ridge(**parameters)

model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
score = r2_score(Y_test, Y_pred)
print("R2 {}".format(score))
mse = mean_squared_error(Y_test, Y_pred)
print("MSE", mse)

R2 0.4130949185868399
MSE 6593.204095494041
